# Problema del commesso viaggiatore simmetrico (TSP)

## Variabili di decisione

$x_{ij} (i, j) \in A$ variabile binaria uguale ad 1 se l'arco $(i, j)$ appartiene al circuito hamiltoniano, 0 altrimenti.

## Funzione obiettivo

Minimizzare il costo totale del circuito hamiltoniano

\begin{equation}
\text{min} \quad \sum_{(i,j) \in A} d_{ij} \cdot x_{ij}
\tag{0}
\end{equation}

## Vincoli

- **Vincoli di assegnamento** In un circuito hamiltoniano ogni nodo deve avere esattamente un arco entrante ed esattamente un arco uscente.

\begin{equation}
\sum_{i: (i,j) \in A} x_{ij} = 1 \quad j \in V
\tag{1}
\end{equation}

\begin{equation}
\sum_{i: (j,i) \in A} x_{ji} = 1 \quad j \in V
\tag{2}
\end{equation}

- **Vincoli di assenza di sottogiri** In un circuito hamiltoniano non possono esserci cicli su un sottoinsieme proprio dell'insieme dei nodi $V$.

\begin{equation}
\sum_{i \in S , j \in S : (i,j) \in A} x_{ij} \leq |S|-1 \quad S \subset V : 2 \leq |S| \leq |V|-1
\tag{3}
\end{equation}


In [1]:
def leggi_coordinate(path):
    indice_linea_dimensione = 4
    indice_inizio_linee_coordinate = 9
    coordinate = []
    with open(path, "r") as f:
        linee = f.readlines()
        linee[indice_linea_dimensione - 1].strip('\n')
        data = linee[indice_linea_dimensione - 1].split(':')
        dimensione = int(data[1])
        for i in range(0, dimensione):
            data = linee[indice_inizio_linee_coordinate - 1 + i].split()
            coordinata = [float(data[1]),float(data[2])]
            coordinate.append(coordinata)
    return coordinate, dimensione

In [2]:
coordinate, dimensione = leggi_coordinate("burma14.txt")

In [3]:
import math

def calcolo_distanza(nodo1,nodo2):
    n1, n2 = coordinate[nodo1], coordinate[nodo2]
    return math.sqrt( ((n1[0]-n2[0]) ** 2) + ((n1[1]-n2[1]) ** 2) )

In [4]:
nodi = [i for i in range(dimensione)]
distanze = {(n1,n2): calcolo_distanza(n1, n2) for n1 in nodi for n2 in nodi if n1 != n2}

In [5]:
import gurobipy as gp
from gurobipy import GRB

mod = gp.Model('TSP')
vars = mod.addVars(distanze.keys(), obj = distanze, vtype=GRB.BINARY, name = 'x')

Academic license - for non-commercial use only - expires 2021-07-17
Using license file C:\Users\hp\gurobi.lic


In [6]:
v1 = mod.addConstrs(vars.sum('*',j) == 1 for j in nodi)
v2 = mod.addConstrs(vars.sum(j,'*') == 1 for j in nodi)

In [7]:
# Da implementare subtour
def calcola_sottogiro_minore(archi):
    unvisited = nodi[:]
    cycle = nodi[:]
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in archi.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle
    return cycle

In [8]:
def eliminazione_sottogiri(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
        tour = calcola_sottogiro_minore(selected)
        if len(tour) < len(nodi):
            model.cbLazy(gp.quicksum(model._vars[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

In [9]:
mod._vars = vars
mod.Params.lazyConstraints = 1
mod.optimize(eliminazione_sottogiri)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 28 rows, 182 columns and 364 nonzeros
Model fingerprint: 0x7890c851
Variable types: 0 continuous, 182 integer (182 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 28 rows, 182 columns, 364 nonzeros
Variable types: 0 continuous, 182 integer (182 binary)

Root relaxation: objective 2.667132e+01, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      30.8785039   30.87850  0.00%     -    0s

Cutting planes:
  Lazy constraints: 5

Explored 0 nodes (

In [10]:
vals = mod.getAttr('x',vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = calcola_sottogiro_minore(selected)
assert len(tour) == len(nodi)

In [11]:
tour

[0, 9, 8, 10, 7, 12, 6, 11, 5, 4, 3, 2, 13, 1]